In [ ]:
%pip install seaborn
%pip install matplotlib
%pip install xgboost


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")
import xgboost as xgb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
Categories=['Mild_Demented','Moderate_Demented','Very_Mild_Demented','Non_Demented']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='/Users/annabelleyao/Desktop/MRIdataset'
#path which contains all the categories of images
for i in Categories:
	print(f'loading... category : {i}')
	path=os.path.join(datadir,i)
	for img in os.listdir(path):
		img_array=imread(os.path.join(path,img))
		img_resized=resize(img_array,(224,224,3))
		img_resized = np.transpose(img_resized, (2, 0, 1))
		flat_data_arr.append(img_resized.flatten())
		target_arr.append(Categories.index(i))
	print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)


loading... category : Mild_Demented
loaded category:Mild_Demented successfully
loading... category : Moderate_Demented
loaded category:Moderate_Demented successfully
loading... category : Very_Mild_Demented
loaded category:Very_Mild_Demented successfully
loading... category : Non_Demented
loaded category:Non_Demented successfully


In [14]:
#dataframe
df=pd.DataFrame(flat_data)
df['Target']=target
df.shape


(6400, 150529)

In [15]:
#input data
x=df.iloc[:,:-1]
#output data
y=df.iloc[:,-1]


In [16]:
# Splitting the data into training and testing sets
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,shuffle = False)
import numpy as np

# Suppose X is your data and y are your labels

# And these are your predefined indices for training and testing
with open("/Users/annabelleyao/Desktop/train_indices.txt", "r") as f:
    train_indices = [int(idx.strip()) for idx in f.readlines()]
with open("/Users/annabelleyao/Desktop/test_indices.txt", "r") as f:
    test_indices = [int(idx.strip()) for idx in f.readlines()]

# You can use these indices to split your data
x_train, y_train = x.iloc[train_indices], y.iloc[train_indices]
x_test, y_test = x.iloc[test_indices], y.iloc[test_indices]

print("Training data:", x_train.shape)
# print("Training labels:", y_train[:10])
print("Testing data:", x_test.shape)
# print("Testing labels:", y_test)

Training data: (5120, 150528)
Testing data: (1280, 150528)


In [17]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10],
			'gamma':[0.1],
			'kernel':['rbf']}

# Creating a support vector classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

# Fit the classifier on the training data
xgb_classifier.fit(x_train, y_train)

# Make predictions on the test data
y_pred = xgb_classifier.predict(x_test)

# Calculate accuracy on the test data
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)


Accuracy: 0.98203125


In [18]:
print(classification_report(y_test, y_pred, target_names=['Mild_Demented','Moderate_Demented','Very_Mild_Demented','Non_Demented']))
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report, auc, recall_score, precision_score, precision_recall_curve

# probabilities = model.predict_proba(x_test)


                    precision    recall  f1-score   support

     Mild_Demented       1.00      0.96      0.98       204
 Moderate_Demented       1.00      0.83      0.91        12
Very_Mild_Demented       0.97      0.98      0.98       430
      Non_Demented       0.98      0.99      0.99       634

          accuracy                           0.98      1280
         macro avg       0.99      0.94      0.96      1280
      weighted avg       0.98      0.98      0.98      1280



In [21]:
probabilities = xgb_classifier.predict_proba(x_test)
# Calculate the ROC AUC score for each class
roc_auc_scores = []
for class_idx in range(probabilities.shape[1]):
    class_true_labels = (y_test == class_idx).astype(int)
    class_predicted_probs = probabilities[:, class_idx]
    roc_auc = roc_auc_score(class_true_labels, class_predicted_probs)
    roc_auc_scores.append(roc_auc)

# Calculate the average ROC AUC score
avg_roc_auc = np.mean(roc_auc_scores)

# Print the average ROC AUC score
print("Average ROC AUC Score:", avg_roc_auc)

Average ROC AUC Score: 0.9990408141217063


In [20]:
# path='/Users/annabelleyao/Desktop/MRIdataset'
# img=imread(path)
# plt.imshow(img)
# plt.show()
# img_resize=resize(img,(150,150,3))
# l=[img_resize.flatten()]
# probability=model.predict_proba(l)
# for ind,val in enumerate(Categories):
# 	print(f'{val} = {probability[0][ind]*100}%')
# print("The predicted image is : "+Categories[model.predict(l)[0]])
